In [1]:
import os
import random
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

class Cutout(object):
    """從圖像中隨機遮蓋一個或多個區域。
    Args:
        n_holes (int): 每個圖像要遮蓋的區域數量。
        length (int): 每個正方形區域的邊長（以像素為單位）。
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img, label):
        """
        Args:
            img (PIL.Image): 輸入圖像。
            label (list): 圖像的標籤，每個標籤是一個列表 [class, x_center, y_center, width, height]。
        Returns:
            PIL.Image: 被遮蓋過的圖像，其中有 n_holes 個尺寸為 length x length 的區域被遮蓋。
            list: 經過 Cutout 轉換後的新標籤。
        """
        w, h = img.size

        mask = np.ones((h, w), np.float32)
        new_label = []

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0.

            # 檢查標籤是否在遮蓋區域內，如果是則不加入新標籤
            for obj in label:
                x_center = float(obj[1]) * w
                y_center = float(obj[2]) * h
                width = float(obj[3]) * w
                height = float(obj[4]) * h

                if x_center >= x1 and x_center <= x2 and y_center >= y1 and y_center <= y2:
                    continue

                new_label.append(obj)

        mask = torch.from_numpy(mask)
        img = transforms.ToTensor()(img)
        img = img * mask

        return transforms.ToPILImage()(img), new_label

def main(input_folder, output_folder, n_holes, length):
    # 如果輸出文件夾不存在，則創建它
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 獲取輸入文件夾中的所有圖片文件
    image_files = [filename for filename in os.listdir(input_folder) if filename.endswith(".png")]
    # 打亂圖片文件列表，以隨機順序處理圖片
    random.shuffle(image_files)

    # 定義 Cutout 轉換
    cutout_transform = Cutout(n_holes=n_holes, length=length)

    # 遍歷輸入文件夾中的圖片文件
    for filename in image_files:
        # 打開圖像
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)

        # 載入原始標籤
        label_path = os.path.join(input_folder, filename.replace(".png", ".txt"))
        original_label = load_original_label(label_path)

        # 應用 Cutout 轉換
        cutout_img, new_label = cutout_transform(img, original_label)

        # 保存被遮蓋過的圖像，文件名以 "cutout_" 為前綴
        cutout_filename = f"cutout_{filename}"
        cutout_img.save(os.path.join(output_folder, cutout_filename))

        # 生成並保存新的標籤
        new_label_path = os.path.join(output_folder, f"cutout_{filename.replace('.png', '.txt')}")
        save_label(new_label, new_label_path)

def load_original_label(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    return [line.strip().split() for line in lines]

def save_label(label, label_path):
    with open(label_path, 'w') as f:
        for obj in label:
            f.write(" ".join([str(x) for x in obj]) + "\n")

# 示例用法:
input_folder = r"C:\Users\Ray\Desktop\宜鼎\c4_second_slice"
output_folder = "Cutout"
n_holes = 1
length = 500  # 根據需要調整長度
main(input_folder, output_folder, n_holes, length)
